In [8]:
import os
import requests
import shutil
import re

dirName = "announcements"
dirPath = ".\\" + dirName + "\\"

try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  announcements  already exists


In [9]:
dirPath

'.\\announcements\\'

In [19]:
def download_file(url):
    file = requests.get(url, stream=True)
    headers = file.headers['content-disposition']
    print(headers)
    filename = re.findall("filename=\"(.+)\"", headers)[0]
    print(filename)
    location = os.path.relpath(dirPath + filename)
    print(location)
    with open(location, 'wb') as location:
        shutil.copyfileobj(file.raw, location)
    del file

In [11]:
from googlesearch import search 
query = "\"S&P 500 INDEX –\" \"Set to Join\" site:us.spindices.com filetype:pdf"
results = search(query, tld='com', lang='en', num=10, start=0, stop=None, pause=2.0)

In [20]:
for result in results:
    download_file(result)
    print(result)

attachment; filename="948566_mktxaaxnmlab546usna6xncr6.pdf"
948566_mktxaaxnmlab546usna6xncr6.pdf
announcements\948566_mktxaaxnmlab546usna6xncr6.pdf
https://us.spindices.com/documents/indexnews/announcements/20190624-948566/948566_mktxaaxnmlab546usna6xncr6.pdf?force_download=true
attachment; filename="1098962_Final5Ingersol4Cimar.pdf"
1098962_Final5Ingersol4Cimar.pdf
announcements\1098962_Final5Ingersol4Cimar.pdf
https://us.spindices.com/documents/indexnews/announcements/20200227-1098962/1098962_Final5Ingersol4Cimar.pdf?force_download=true
attachment; filename="935564_dwdpspin5blmn6.pdf"
935564_dwdpspin5blmn6.pdf
announcements\935564_dwdpspin5blmn6.pdf
https://us.spindices.com/documents/indexnews/announcements/20190528-935564/935564_dwdpspin5blmn6.pdf?force_download=true
inline; filename="11700_3dmaccor546.pdf"
11700_3dmaccor546.pdf
announcements\11700_3dmaccor546.pdf
https://us.spindices.com/ids_spice_data/idsfiles/11700_3dmaccor546.pdf
inline; filename="20171228-spdji-bard-huntington-